In [3]:
import numpy as np
import pandas as pd
import joblib
from sklearn.metrics import recall_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold, StratifiedKFold

In [2]:
train_data = pd.read_csv('../../data/raw/train.csv',  encoding= 'unicode_escape')
y_train = train_data["Unusual"]                      #defining the labels
X_train = train_data.drop(["Unusual"], axis=1)
test_data = pd.read_csv('../../data/raw/test.csv',  encoding= 'unicode_escape')
y_test = test_data["Unusual"]                      #defining the labels
X_test = test_data.drop(["Unusual"], axis=1)

In [4]:
# Define the model pipeline
model_pipeline = Pipeline([
    ('preprocessing', joblib.load('../pipelines/PreprocessingPipeline.joblib')),
    ('xgboost', XGBClassifier()),    
]) 

# KFold cross-validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Perform cross-validation
scores = cross_val_score(model_pipeline, X_train, y_train, cv=cv)

print(f'Cross-Validation accuracy: {round(scores.mean(),3)} +/- {round(scores.std(),3)}')

Cross-Validation accuracy: 0.987 +/- 0.002


In [6]:
# without cross-validation

#train the model
model_pipeline.fit(X_train,y_train)

# predict the vlaues
pred = model_pipeline.predict(X_test)

#recall
recall_score(y_test,pred)

0.9594775619557937

In [7]:
model_pipeline.get_params

<bound method Pipeline.get_params of Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Unnamed: 0', 'PRBUsageUL',
                                                   'PRBUsageDL', 'meanThr_DL',
                                                   'meanThr_UL', 'maxThr_DL',
                                                   'maxThr_UL', 'meanUE_DL',
                                                   'meanUE_UL', 'maxUE_DL',
                                                   'maxUE_UL']),
                                                 ('cat',
                